# Search Pubmed

## Define Search Criteria

In [1]:
from Bio import Entrez
import numpy as np

def search(query):
    Entrez.email = 'your.email@example.com'
    handle = Entrez.esearch(db='pubmed', 
                            sort='most recent', 
                            retmax='5000',
                            retmode='xml', 
                            reldate = 7, #only within n days from now
                            term=query)
    results = Entrez.read(handle)
    return results

#search terms (can test string with Pubmed Advanced Search)
search_results = search('(Biomech*[Title/Abstract] OR locomot*[Title/Abstract])')

## Perform Search and Save Paper Titles

In [2]:


def fetch_details(id_list):
    ids = ','.join(id_list)
    Entrez.email = 'your.email@example.com'
    handle = Entrez.efetch(db='pubmed',
                           retmode='xml',
                           id=ids)
    results = Entrez.read(handle)
    return results

id_list = search_results['IdList']
papers = fetch_details(id_list)
print("")
titles = [0 for i in enumerate(papers['PubmedArticle'])]
keywords = ['' for i in enumerate(papers['PubmedArticle'])]
authors = ['' for i in enumerate(papers['PubmedArticle'])]
links = ['' for i in enumerate(papers['PubmedArticle'])]
journals = ['' for i in enumerate(papers['PubmedArticle'])]


for i, paper in enumerate(papers['PubmedArticle']):
    titles[i] = papers['PubmedArticle'][i]['MedlineCitation']['Article']['ArticleTitle']
print(np.size(titles),'Papers found')


122 Papers found


## Pull information from PubMed Results
#### Format title, journal, authors in markdown friendly manner

In [3]:
for i, paper in enumerate(papers['PubmedArticle']):
    links[i] = "* [%s](https://www.ncbi.nlm.nih.gov/pubmed/%s)" % (paper['MedlineCitation']['Article']['ArticleTitle'],paper['MedlineCitation']['PMID'])
    auths = []
    for auth in paper['MedlineCitation']['Article']['AuthorList']:
        try:
            auth_name = [auth['LastName'],auth['Initials']+',']
            auth_name = ' '.join(auth_name)
            auths.append(auth_name)
        except:
            print(paper['MedlineCitation']['Article']['ArticleTitle'],'- NAME ERROR')
    authors[i] = ' '.join(auths)
    journals[i] = '*%s*' % (paper['MedlineCitation']['Article']['Journal']['Title']) 
    #store keywords 
    if paper['MedlineCitation']['KeywordList'] != []:
        kwds = []
        for kw in paper['MedlineCitation']['KeywordList'][0]:
            kwds.append(kw[:])         
        keywords[i] = ' '.join(kwds)

### Markdown Example:

* [Skeletal muscles of hibernating black bears show minimal atrophy and phenotype shifting despite prolonged physical inactivity and starvation.](https://www.ncbi.nlm.nih.gov/pubmed/30998788)
Miyazaki M,
Shimozuru M,
Tsubota T,
*PloS one*
<br>  

* [Phase space methods for non-linear analysis of pedalling forces in cycling.](https://www.ncbi.nlm.nih.gov/pubmed/30998746)
Kunert A,
Ott M,
Reuter T,
Koska D,
Maiwald C,
*PloS one*
<br>  

# Clean up title and keyword strings

In [ ]:
print('Number of Papers: ',np.size(titles,0)) #number of papers. limited to 500. (retmax)
#clean up titles
import re
titles = [t.lower() for t in titles] #same case
titles = [t.replace('<sub>',' ').replace('</sub>','') for t in titles] #subscript
titles = [t.replace('<i>',' ').replace('</i>','') for t in titles] #italics
titles = [t.replace('[','').replace(']','') for t in titles] #remove brackets from html parser
#clean up keywords
keywords = [k.lower() for k in keywords] #same case


Number of Papers:  122


# Load Top-performing Model

In [ ]:
from keras.models import model_from_json
# load json and create model
json_file = open('Models/keras_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("Models/keras_model.h5")
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
print("\nLoaded model from disk")
 

Using TensorFlow backend.


# Load Associated Vectorizer

In [ ]:
import pickle
# import numpy as np
from sklearn.preprocessing import LabelEncoder
#load vectorizer and label encoder
vect = pickle.load(open('Models/keras_Vectorizer.pkl','rb'))
le = LabelEncoder()
le.classes_   = np.load('Models/keras_LabelEncoder.npy')
print('\nLoaded Vectorizer')


# Vectorize Strings

In [ ]:
#get titles for this week's literature update
import pandas as pd
papers_df = pd.DataFrame({'title': titles, 'keywords': keywords})
#join keywords with titles
papers_df['everything'] = papers_df['title'] + ' ' + papers_df['keywords']

#vectorize 
titles_vec = vect.transform(papers_df['everything'])
#OR if you don't want to use just the title:
# titles_vec = vect.transform(papers_df['title'])

# Predict Topics For Each Paper

In [ ]:
prediction_vec = model.predict(titles_vec)
topics = [le.inverse_transform([np.argmax(top_val)])[0] for top_val in model.predict(titles_vec)]
papers_df['topic'] = topics
papers_df[['title','topic']].head()

# Save Paper Titles and Topics

### Store everything in DataFrame and sort by Topic

In [ ]:
#add info for github markdown format
papers_df['authors'] = authors
papers_df['journal'] = journals
papers_df['links'] = links
#generate filename
import datetime
now = datetime.datetime.now()
strings = [str(now.year), str(now.month), str(now.day),'litupdate.csv']
fname = 'Literature_Updates/'+'-'.join(strings)
strings = [str(now.year), str(now.month), str(now.day),'litupdate.md']
mdname = 'Literature_Updates/'+'-'.join(strings)
strings = [str(now.year), str(now.month), str(now.day),'litupdate']
urlname = '-'.join(strings)

print('Filename: ',fname)

# papers_df = papers_df.sort_values(by = ['topic'])
# papers_df = papers_df.reset_index(drop = True)
papers_df.head()

### Save as .csv 

In [ ]:
# header = ['title','topic']
# papers_df.sort_values('topic').to_csv(fname, index = False, columns = header)
# print('\nLiterature Update Exported as .csv')


# Compile papers grouped by topic

In [ ]:
md_file = open(mdname, 'w', encoding = 'utf-8')
md_file.write('---\n')
md_file.write('title: Biomechanics Literature Update\n')
md_file.write('collection: literature\n')
md_file.write('permalink: /literature/%s\n' % urlname)
md_file.write('excerpt: <br>\n')
md_file.write('---\n')


#tidy up topic strings
topic_list = np.unique(papers_df.sort_values('topic')['topic'])
ss = [s for s in topic_list if 'UNIQUE' in s]
for i,t in enumerate(topic_list):
    if 'UNIQUE' in t:  
        topic_list[i] = 'UNIQUE TOPIC'
    if 'IMPACT' in t:
        topic_list[i] = 'TRAUMA/IMPACT'

md_file.write('### Created by: [Ryan Alcantara](https://twitter.com/Ryan_Alcantara_) & [Gary Bruening](https://twitter.com/garebearbru) - University of Colorado Boulder\n')
md_file.write('### Table Of Contents: \n')
for topic in topic_list:
    md_file.write('[%s](#%s)  \n' % (topic, str.lower(topic).replace('/','').replace(' ','')))
md_file.write('\n')
for topic in topic_list:
    md_file.write('----\n')
    md_file.write('# %s\n' % topic)
    md_file.write('----\n')
    md_file.write('\n')
    md_file.write('[Back to top](#table-of-contents)')
    md_file.write('\n')
    papers_subset = pd.DataFrame(papers_df[papers_df.topic == topic].reset_index(drop = True))
    for i,paper in enumerate(papers_subset['links']):
        md_file.write('%s\n' % paper)
        md_file.write('%s\n' % papers_subset['authors'][i])
        md_file.write('%s.  \n' % papers_subset['journal'][i])
        md_file.write('\n')

md_file.close()
print('Literature Update Exported as Markdown')
print('Location:',mdname)